## Diagnostic Business Cycles

- Author: yuxuan zhao
- Last Update: 2024/5/13

## Figure 1: PIH model under DE with naivete

## Model

- quadratic utility and iid income shocks. 

- capital price $q=(1+r)^{-1}$, 

- $r>0$ is the exogenous real interest rate

- discount factor is $\beta=(1+r)^{-1}$. 


The time $t$ optimaization problem is then

$$

\max _{K_t^\theta}\left\{u\left(C_t^\theta\right)+\mathbb{E}_t^\theta\left[\mathcal{V}\left(K_t^\theta\right)\right]\right\}

$$

The naivete assumption is that the agent at time $t$ believes his future selves from time $t+1$ on will act under RE. 

Thus, the agent expects that the time $t+1$ self will take the action $K_{t+1}^{R E}$ so as

$$

\mathcal{V}\left(K_t^\theta\right)=\max _{K_{t+1}^{R E}}\left[u\left(C_{t+1}^{R E}\right)+\mathbb{E}_{t+1}\left[\mathcal{V}\left(K_{t+1}^{R E}\right)\right]\right]

$$

with budget constraint at time $t$: 

$$

K_t^\theta=(1+r)\left(K_{t-1}^\theta+\bar{Y}+\varepsilon_t-C_t^\theta\right)

$$

## Analytical solution in appendix:

\begin{align*}

K_t^\theta &= \frac{1}{1+r(1+\theta)}\left[(1+r)K_{t-1}^\theta+r \theta K_{t-J}^\theta+(1+r) \varepsilon_t\right] \\

C_t^\theta &= K_{t-1}^\theta+\bar{Y}+\varepsilon_t-\frac{K_t^\theta}{1+r}

\end{align*}

In [ ]:
# Import all libraries
import numpy as np
from numpy.random import randn
import scipy
from scipy.interpolate import RectBivariateSpline, interpn, interp1d
from scipy.linalg import qz, svd, inv
import quantecon as qe
import pandas as pd
import matplotlib.pyplot as plt
import copy
import sympy as sp
import math
use_latex='mathjax'

In [ ]:
# Define IRF function
def irf_solve(theta, J, r, Y, sigma, T):
    K = np.zeros((T+1,1)) # capital path
    C = np.zeros((T+1,1)) # consumption path
    KS = np.zeros((T+1,1)) # capital surprise path
    # income shock path
    epsilon = np.zeros((T+1,1)) 
    epsilon[1] = sigma

    # initial condition
    K[0] = 0
    C[0] = 0
    KS[0] = 0

    # 计算动态系统的演化
    for t in range(1, T+1):
        K[t] = (1 / (1 + r * (1 + theta))) * ((1+r) * K[t-1] + r * theta * K[max(t-J, 0)] + (1 + r) * epsilon[t])
        C[t] = K[t-1] + Y + epsilon[t] - K[t] / (1 + r)
        KS[t] = theta*(K[t-1] - K[max(t-J, 0)])

    #remove the initial point
    K = K[1:] # capital path
    C = C[1:] # consumption path
    KS = KS[1:] # capital surprise path

    return K, C, KS

# 定义参数
J_all=np.array([1,2,5,8])
theta_all=np.array([0,1,2,5])

#theta = 0
#J = 5
r = 0.05
Y = 0 #i.i.d income process
sigma = 1
T = 20


# Create a 4x4 grid of subplots
fig, axs = plt.subplots(3, 4, figsize=(10, 6))

# Iterate through each combination of J and theta
for j, J_val in enumerate(J_all):
    for i, theta_val in enumerate(theta_all):
        theta = theta_val
        J = J_val
        K, C, KS = irf_solve(theta, J, r, Y, sigma, T)
        path = np.array([K, C, KS])
        for m, P in enumerate(path):
            ax = axs[m, j]
            ax.plot(np.arange(T), P)


# Add row and column annotations
#for i, j_val in enumerate(J_all):
    #fig.text(0.1, 0.5 * (i + 0.5) / len(J_all), f'J={j_val}', ha='center', va='center')


plt.tight_layout()
plt.show()

## Solve linear RE model (Sims2002)

In [ ]:
def qzswitch(i, A, B, Q, Z):
    '''
    Takes U.T. matrices A, B, orthonormal matrices Q,Z, interchanges diagonal elements i and i+1 of both A and B, while maintaining Q'AZ' and Q'BZ' unchanged. 
    Does nothing if ratios of diagonal elements in A and B at i and i+1 are the same.  
    Aborts if diagonal elements of both A and B are zero at either position.
    '''
    a, d, b, e, c, f = A[i, i], B[i, i], A[i, i+1], B[i, i+1], A[i+1, i+1], B[i+1, i+1]
    #A(i:i+1,i:i+1)=[a b; 0 c]
	#B(i:i+1,i:i+1)=[d e; 0 f]

    wz = np.array([c*e-f*b, c*d-f*a]) #缩放矩阵
    xy = np.array([b*d-e*a, c*d-f*a]) #旋转矩阵

    n = np.linalg.norm(wz)
    m = np.linalg.norm(xy)

    if n == 0:
        return A, B, Q, Z

    else:
        wz /= n
        xy /= m
        wz = np.vstack((wz, [-wz[1], wz[0]]))
        xy = np.vstack((xy, [-xy[1], xy[0]]))

        A[i:i+2, :] = np.dot(xy, A[i:i+2, :])
        B[i:i+2, :] = np.dot(xy, B[i:i+2, :])
        A[:, i:i+2] = np.dot(A[:, i:i+2], wz)
        B[:, i:i+2] = np.dot(B[:, i:i+2], wz)
        Z[:, i:i+2] = np.dot(Z[:, i:i+2], wz)
        Q[i:i+2, :] = np.dot(xy, Q[i:i+2, :])

        return A, B, Q, Z



In [ ]:

def qzdiv(stake, A, B, Q, Z):
    '''
    Takes U.T. matrices A, B, orthonormal matrices Q,Z, rearranges them so that all cases of abs(B(i,i)/A(i,i))>stake are in lower right corner
    while preserving U.T. and orthonormal properties and Q'AZ' and Q'BZ'.
    '''

    #stake (float): 阈值，用于判断是否需要重新排列对角线元素
    n, _ = A.shape
    root = abs(np.vstack((np.diag(A), np.diag(B))))
    root[0, :] -= (root[0, :] < 1.e-13) * (root[0, :] + root[1, :])
    root[1, :] /= root[0, :]

    for i in range(n, 0, -1):
        m = 0
        for j in range(i, 0, -1):
            if root[1, j-1] > stake or root[1, j-1] < -0.1:
                m = j
                break
        if m == 0:
            return A, B, Q, Z
        for k in range(m-1, i-1):
            A, B, Q, Z = qzswitch(k, A, B, Q, Z)
            tmp = root[1, k-1]
            root[1, k-1] = root[1, k]
            root[1,k] = tmp

    return A, B, Q, Z



In [ ]:
def gensysct(g0, g1, c, psi, pi, div=None):
    """
    System given as
    g0*y(t)=g1*y(t-1)+c+psi*z(t)+pi*eta(t),
    with z(t) an exogenous variable process and eta(t) being endogenously determined expectational errors.  
    Returned system as
    Dy(t)=G1*y(t)+C+impact*epsilon(t).
    z(t) is assumed to be i.i.d
    eu(1)=1 for existence, eu(2)=1 for uniqueness.  eu(1)=-1 for existence only with not-s.c. z; eu=[-2,-2] for coincident zeros.
    """
    
    tol = np.sqrt(np.finfo(float).eps) * 10
    eu = np.zeros(2)
    fixdiv = (div is not None)

    n = g0.shape[0]
    a, b, q, z = qz(g0, g1)

      #div是分辨特征值矩阵特征值稳定与否的变量，由于数值误差的存在，我们先假定div=1.01
    if not fixdiv:
        div = 1.01
    
    nunstab = 0 #稳定特征值的数量
    zxz = False
    for i in range(n):
        #在没有外部给定div值的情况下，我们根据特征值的比率调整div的大小
        if not fixdiv:
            if abs(a[i, i]) > 0:
                divhat = abs(b[i, i]) / abs(a[i, i])
                if 1 + tol < divhat <= div:
                    div = 0.5 * (1 + divhat)
        nunstab += abs(b[i, i]) > div * abs(a[i, i])
        #如果特征值在1+tol和div之间，我们缩小div使得特征值大于div
        if abs(a[i, i]) < tol and abs(b[i, i]) < tol:
            zxz = True

    #如果矩阵满秩，则直接对a, b矩阵做qz分解,否则无法解线性系统 
    if not zxz:
        a, b, q, z = qzdiv(div, a, b, q, z)
    else:
        eu[:] = -2
        return None, None, None, None, None, None, None, eu, None
        raise ValueError("Indeterminacy: The linear system has coincident zeros.")

    gev = np.hstack((np.diag(a), np.diag(b)))

    q1 = q[:n - nunstab, :]
    q2 = q[n - nunstab:, :]
    z1 = z[:, :n - nunstab].T
    z2 = z[:, n - nunstab:].T
    a2 = a[n - nunstab:, n - nunstab:]
    b2 = b[n - nunstab:, n - nunstab:]

    # Compute singular value decomposition of pi and psi
    etawt = np.dot(q2, pi)
    zwt = np.dot(q2, psi)
    ueta, deta, veta = svd(etawt)
    deta = np.diag(deta)
    md = min(deta.shape)
    bigev = np.where(np.diag(deta[:md]) > tol)[0]
    ueta = ueta[:, bigev]
    veta = veta[:, bigev]
    deta = deta[np.ix_(bigev,bigev)]
    uz, dz, vz = svd(zwt)
    dz = np.diag(dz)
    md = min(dz.shape)
    bigev = np.where(np.diag(dz[:md]) > tol)[0]
    uz = uz[:, bigev]
    vz = vz[:, bigev]
    dz = dz[np.ix_(bigev,bigev)]

    if bigev.size == 0:
        exist = 1
    else:
        exist = np.linalg.norm(uz - np.dot(ueta, np.dot(ueta.T, uz)), 'fro') < tol * n
    # Check existence of solutions for non-zero singular values
    if bigev.size != 0:
        zwtx0 = np.linalg.solve(b2, zwt)
        zwtx = zwtx0
        M = np.linalg.solve(b2, a2)
        M = M / np.linalg.norm(M)
        for i in range(1, nunstab):
            zwtx = np.hstack((M @ zwtx, zwtx0))
        zwtx = b2 @ zwtx
        ux, dx, vx = svd(zwtx)
        dx = np.diag(dx)
        md = min(dx.shape)
        bigev = np.where(np.diag(dx[:md]) > tol)[0]
        ux = ux[:, bigev]
        vx = vx[:, bigev]
        dx = dx[np.ix_(bigev,bigev)]
        existx = np.linalg.norm(ux - np.dot(ueta, np.dot(ueta.T, ux)), 'fro') < tol * n
    else:
        existx = 1
    # Compute singular value decomposition of q1*pi
    ueta1, deta1, veta1 = svd(np.dot(q1, pi))
    deta1 = np.diag(deta1)
    md = min(deta1.shape)
    bigev = np.where(np.diag(deta1[:md]) > tol)[0]
    ueta1 = ueta1[:, bigev]
    veta1 = veta1[:, bigev]
    deta1 = deta1[np.ix_(bigev,bigev)]
    # Check existence and uniqueness of solutions
    if existx or nunstab == 0:
        #solution exists
        eu[0] = 1
    else:
        if exist:
            #solution exists for unforecastable z only
            eu[0] = -1
        else:
            return None, None, None, None, None, None, None, eu, None
            raise ValueError("No solution: The linear system has unstable roots. endog errors.")
        
    if veta1.size == 0:
        #unique solution exists
        unique = 1
    else:
        #unique solution exists
        unique = np.linalg.norm(veta1 - np.dot(veta, np.dot(veta.T, veta1)), 'fro') < tol * n
    if unique:
        eu[1] = 1
    # Compute transformation matrix tmat
    # 计算 tmat
    tmat = np.hstack((np.eye(n - nunstab), -np.dot(ueta1, np.dot(deta1, np.dot(veta1.T, np.dot(veta, np.linalg.solve(deta, ueta.T)))))))
    # Construct matrices G0 and G1
    G0 = np.vstack((np.dot(tmat, a), np.hstack((np.zeros((nunstab, n - nunstab)), np.eye(nunstab)))))
    G1 = np.vstack((np.dot(tmat, b), np.zeros((nunstab, n))))
    # Compute constant term C
    G0I = inv(G0)
    G1 = np.dot(G0I, G1)
    usix = np.arange(n-nunstab, n)
    C = np.dot(G0I, np.vstack((np.dot(tmat, np.dot(q, c)), np.linalg.solve(a[np.ix_(usix, usix)] - b[np.ix_(usix, usix)], np.dot(q2, c)))))
    # Compute impact matrix
    impact = np.dot(G0I, np.vstack([np.dot(tmat, np.dot(q, psi)), np.zeros((nunstab, psi.shape[1]))]))
    # Compute G1 in terms of original coordinates
    G1 = np.dot(np.dot(z, G1), z.T)
    # Check for accuracy in G1
    if np.max(np.abs(np.imag(G1))) > 100 * tol:
        print('Inaccuracy in G1:')
        s1 = svd(G1)
        s2 = svd(np.real(G1))
        print(np.max((s1 - s2) / (1 / 12 + s1)))  # this is reasonable scaling for monthly time unit
    # Return real parts of G1, C, and impact
    G1 = np.real(G1)
    C = np.real(np.dot(z, C))
    impact = np.real(np.dot(z, impact))
    return G1, C, impact, q, a, b, z, eu

## A simple example to illustrate the algorithm
In this part, we depict method of Sims (2002) in a simple example. We consider the following univariate model, variant of an example studied in Iskrev (2008) :
$$
\theta^2 \kappa z_t=\theta^2 \mathbb{E}_t z_{t+1}+(\theta \kappa-1) z_{t-1}+\varepsilon_t
$$

where $0<\theta<1$ and $\kappa>0$ .

We define $y_t=\mathbb{E}_t z_{t+1}$, and $\eta_t=z_t-y_{t-1}$. The model is then rewritten as:
$$
\left[\begin{array}{cc}
\frac{\theta^2 \kappa}{\theta \kappa-1} & \frac{-\theta^2}{\theta \kappa-1} \\
1 & 0
\end{array}\right]\left[\begin{array}{l}
z_t \\
y_t
\end{array}\right]=\left[\begin{array}{c}
z_{t-1} \\
y_{t-1}
\end{array}\right]+\left[\begin{array}{c}
{\theta \kappa-1} \\
0
\end{array}\right] \varepsilon_t+\left[\begin{array}{c}
0 \\
1
\end{array}\right] \eta_t
$$

We manually perform eigenvalue decomposition, where the eigenvalues $\lambda$ satisfy the following conditions:
$$
\left|\lambda \mathbb{I} - \left[\begin{array}{cc}
\frac{\theta^2 \kappa}{\theta \kappa-1} & \frac{-\theta^2}{\theta \kappa-1} \\
1 & 0
\end{array}\right]\right| = 0
$$

$$
(\theta \kappa -1)\lambda^2 - \theta^2 \kappa \lambda + \theta^2 =0
$$

$$
((\theta \kappa -1)\lambda - \theta)(\lambda - \theta) =0
$$

$$
\left[\begin{array}{cc}
\frac{\theta^2 \kappa}{\theta \kappa-1} & \frac{-\theta^2}{\theta \kappa-1} \\
1 & 0
\end{array}\right] = \mathbb{Z} \left[\begin{array}{cc}
\frac{\theta}{\theta \kappa - 1} & 0 \\
0 & \theta
\end{array}\right] \mathbb{Z}^{-1}
$$

When $\frac{\theta}{\theta \kappa-1}>1$, the matrix on the left-hand-side of the former equality has a unique eigenvalue smaller than $1(\theta)$.

$$
\mathbb{Z}^{-1}\left[\begin{array}{l}
z_t \\
y_t
\end{array}\right]=\left[\begin{array}{cc}
\frac{\theta \kappa - 1}{\theta} & 0 \\
0 & \frac{1}{\theta}
\end{array}\right]\mathbb{Z}^{-1}\left[\begin{array}{c}
z_{t-1} \\
y_{t-1}
\end{array}\right]+\left[\begin{array}{cc}
\frac{\theta \kappa - 1}{\theta} & 0 \\
0 & \frac{1}{\theta}
\end{array}\right]\mathbb{Z}^{-1}\left[\begin{array}{c}
{\theta \kappa-1} \\
0
\end{array}\right] \varepsilon_t+\left[\begin{array}{cc}
\frac{\theta \kappa - 1}{\theta} & 0 \\
0 & \frac{1}{\theta}
\end{array}\right]\mathbb{Z}^{-1}\left[\begin{array}{c}
0 \\
1
\end{array}\right] \eta_t
$$

Let $\mathbb{Z}^{-1}\left[\begin{array}{l}
z_t \\
y_t
\end{array}\right]=\left[\begin{array}{l}
m_t \\
n_t
\end{array}\right]$,we have

$$
n_t = \frac{1}{\theta} n_{t-1} + \frac{\theta \kappa - 1}{\theta^2} z_{21} \varepsilon_t + \frac{1}{\theta} z_{22} \eta_t
$$

We already know $\frac{1}{\theta}>1$, then $n_t = z_{21} z_t + z_{22} y_t = 0$ must hold to prevent the state variables of the system from exploding (diverging towards infinity). i.e.

$$
y_t = \frac{z_{21}}{z_{22}} z_t
$$

$$
\frac{\theta \kappa - 1}{\theta^2} z_{21} \varepsilon_t + \frac{1}{\theta} z_{22} \eta_t = 0
$$

i.e.
$$
\eta_t = \frac{(\theta \kappa -1)z_{21}}{\theta z_{22}} \varepsilon_t
$$

Substituting these equations back into the dynamic system, we obtain the solution for the economic system (the law of motion for the state variables).


$$
z_t=\frac{\theta \kappa-1}{\theta} z_{t-1}+\frac{1}{\theta} \varepsilon_t
$$

In [ ]:
#initialization of parameters
theta = 0.9
kappa = 2

# Initialization of matrices
GAM0 = np.zeros((2, 2))
GAM1 = np.zeros((2, 2))
PSI = np.zeros((2, 1))
PPI = np.zeros((2, 1))

#Input dynamic system matrix
GAM0 = np.array([[theta**2*kappa/(theta*kappa -1 ),-theta**2/(theta*kappa -1 )],[1,0]])
GAM1 = np.array([[1,0],[0,1]])
PSI = np.array([[theta*kappa-1],[0]])
PPI = np.array([[0],[1]])


TT, CC, RR, _, _, _, _, RC = gensysct(GAM0,GAM1,0,PSI,PPI)

#print(TT)
#print(TT[:,:1].shape)
#print(TT[:,:1].T.shape)
#print(RR)
#print(RC)

## Using python to do log-linearization
Use the SymPy library for symbolic calculations and the NumPy library for numerical calculations.

RBC model:
- Periodic utility function: $u\left(c_t, 1-h_t\right)=\ln c_t+v_h \ln \left(1-h_t\right), A>0$
- Life-time utility: $\sum_{t=0}^{\infty} \beta^t u\left(c_t, 1-h_t\right), \beta \in(0,1)$
- Production function: $y_t=z_t f\left(k_{t-1}, h_t\right)=z_t k_{t-1}^\alpha h_t^{1-\alpha}, \alpha \in(0,1)$
- LOM of capital: $k_t=(1-\delta) k_{t-1}+i_t, \delta \in(0,1)$
- TFP process: $\ln z_t=\rho \ln z_{t-1}+\varepsilon_t, \varepsilon_t \sim$ i. i. d. $N\left(0, \sigma_{\varepsilon}^2\right), \rho \in(0,1)$

optimization problem:
\begin{aligned}
& \max_{\left\{c_t, h_t, k_t\right\}_{t=0}^{\infty}} E_0 \sum_{t=0}^{\infty} \beta^t\left[\ln c_t+A \ln \left(1-h_t\right)\right] \\
& \text { s.t. } c_t+k_t-(1-\delta) k_{t-1} \leq z_t k_{t-1}^\alpha h_t^{1-\alpha} \\
& \text { States: } k_{t-1}, z_t \\
& \text { Controls: } c_t, h_t, k_t \\
&
\end{aligned}



#### Summary of the nonlinear system
- $1=\beta E_t\left(c_t / c_{t+1}\right)\left[r_{t+1}+(1-\delta)\right]$ intertemporal efficiency condition
- $v_h c_t=\left(1-h_t\right)(1-\alpha) y_t / h_t$ intratemporal efficiency condition
- $c_t+k_t-(1-\delta) k_{t-1}=y_t$ resource constraint
- $y_t=z_t k_{t-1}^\alpha h_t^{1-\alpha}$ production function
- $r_t=\alpha y_t / k_{t-1}$ (implied) real rental rate
- $k_t=(1-\delta) k_{t-1}+i_t$ LOM of capital
- $\ln z_t=\rho \ln z_{t-1}+\varepsilon_t$ TFP process

Seven equations

Seven variables: $k, z, y, c, h, r, i$

In [15]:
# Using python to do log-linearization
# A=[-fxp -fyp], B=[fx fy]
# A * [X_p Y_p] = B * [X Y]
from math import e
# Define parameters
alpha = 0.7
beta = 0.98
delta = 0.15
rho = 0.9
vh = 2

# Define symbolic variables
Css, Hss, Kss, Yss, Rss, Zss, Iss = sp.symbols('Css Hss Kss Yss Rss Zss Iss')
C, H, K, Y, R, Z, I = sp.symbols('C H K Y R Z I')
C_p, H_p, K_p, Y_p, R_p, Z_p, I_p = sp.symbols('C_p H_p K_p Y_p R_p Z_p I_p')

Rss = 1/beta - (1-delta)
Zss = 1
Kss = 1
Css = Rss/(alpha*Kss)-delta*Kss
Hss = Rss/(alpha*Kss**(1+alpha))
Yss = Rss/(alpha*Kss)
Iss = delta*Kss

Rss = math.log(Rss)
Zss = math.log(Zss)
Kss = math.log(Kss)
Css = math.log(Css)
Hss = math.log(Hss)
Yss = math.log(Yss)
Iss = math.log(Iss)

y = [C, H, Y, R, I]
x = [K, Z]
yp = [C_p, H_p, Y_p, R_p, I_p]
xp = [K_p, Z_p]
yss = [Css, Hss, Yss, Rss, Iss]
xss = [Kss, Zss]


# Define equilibrium equations
f = []

# Consumption Choices
f.append(beta * C_p * (R_p + 1 - delta) - 1 / C)

# Labor Choices
f.append((1 - H) * (1 - alpha) * Y / H - vh*C)

# Resource constraint
f.append(C + K_p - (1-delta) * K - Y)

# Production function 
f.append(Z * K ** alpha * H ** (1 - alpha) - Y)

# Real rental rate
f.append(alpha * Y / K - R)

# Law of motion of capital
f.append((1 - delta) * K + I - K_p)

# TFP process
f.append(rho * sp.log(Z) - sp.log(Z_p))

#sp.pprint(f)

f = sp.Matrix(f)
#print(f)

flog = f.subs([(m,e**m) for m in y])
flog = flog.subs([(m,e**m) for m in x])
flog = flog.subs([(m,e**m) for m in yp])
flog = flog.subs([(m,e**m) for m in xp])
#sp.pprint(flog)


fy=[[],[],[],[],[]]
fyp=[[],[],[],[],[]]
fx=[[],[]]
fxp=[[],[]]

for m in range(5):
    fy[m] = sp.diff(flog, y[m]) 
for m in range(5):
    fyp[m] = sp.diff(flog, yp[m]) 
for m in range(2):
    fx[m] = sp.diff(flog, x[m]) 
for m in range(2):
    fxp[m] = sp.diff(flog, xp[m])


#将稳态值代入，首先创建一一对应的字典
dicty = dict(zip(y, yss))
dictyp = dict(zip(yp, yss))
dictx = dict(zip(x, xss))
dictxp = dict(zip(xp, xss))

subs = dicty
subs.update(dictyp)
subs.update(dictx)
subs.update(dictxp)

fy = [expr.subs(subs) for expr in fy]
fyp = [expr.subs(subs) for expr in fyp]
fx = [expr.subs(subs) for expr in fx]
fxp = [expr.subs(subs) for expr in fxp]
#sp.pprint(fy)






## A Quantitative NK Model

## Household: 

The representative household chooses capital $K_t^\theta$, investment $I_t^\theta$, capital utilization rate $u_t^\theta$, bonds $B_t^\theta$, consumption $C_t^\theta$, labour $N_{h, t}^\theta$, and nominal wage $W_{h, t}^\theta$ to solve
$$
\max _{K_t^\theta, I_t^\theta, u_t^\theta, B_t^\theta, C_t^\theta, N_{h, t}^\theta, W_{h, t}^\theta}\left[\ln \left(C_t^\theta-b \bar{C}_{t-1}^\theta\right)-\frac{\left(N_{h, t}^\theta\right)^{1+\eta}}{1+\eta}+\beta \mathbb{E}_t^\theta \mathcal{V}\left(\mathcal{S}_{t+1}^\theta\right)\right]
$$
subject to the budget constraint
$$
\begin{aligned}
& P_t^\theta C_t^\theta+P_t^\theta I_t^\theta+P_t^{B, \theta} B_t^\theta+\left(\varphi_w / 2\right)\left(W_{h, t}^\theta / W_{h, t-1}^\theta-\gamma \Pi\right)^2 W_t^\theta \\
& \quad=B_{t-1}^\theta+P_t^\theta R_t^{k, \theta} u_t^\theta K_{t-1}^\theta+W_{h, t}^\theta N_{h, t}^\theta+\int_0^1 D_{i, t}^\theta d i-P_t^\theta a\left(u_t^\theta\right) K_{t-1}^\theta,
\end{aligned}
$$

the physical capital law of motion, which features a standard quadratic investment adjustment cost
$$
\begin{aligned}
K_t^\theta=(1-\delta) K_{t-1}^\theta+\left\{1-(\kappa / 2)\left(\left(I_t^\theta / I_{t-1}^\theta\right)-\gamma\right)^2\right\} I_t^\theta
\end{aligned}
$$

the household assumes that her and other agents’ future conditional preferences and resulting conditionally optimal actions will be taken under perfect memory (or RE)
$$
\mathcal{V}\left(\mathcal{S}_t^\theta\right)=\operatorname{max}_{K_t^{R E}, I_t^{R E}, u_t^{R E}, B_t^{R E}, C_t^{R E}, N_{h, t}^{R E}, W_{h, t}^{R E}}\left[\ln \left(C_t^{R E}-b \bar{C}_{t-1}^\theta\right)-\frac{\left(N_{h, t}^{R E}\right)^{1+\eta}}{1+\eta}+\beta \mathbb{E}_t \mathcal{V}\left(\mathcal{S}_{t+1}^{R E}\right)\right],
$$
subject to the budget constraint
$$
\begin{aligned}
& P_t^{R E} C_t^{R E}+P_t^{R E} I_t^{R E}+P_t^{B, R E} B_t^{R E}+\left(\varphi_w / 2\right)\left(W_{h, t}^{R E} / W_{h, t-1}^\theta-\gamma \Pi\right)^2 W_t^{R E} \\
& \quad=B_{t-1}^\theta+P_t^{R E} R_t^{k, R E} u_t^{R E} K_{t-1}^\theta+W_{h, t}^{R E} N_{h, t}^{R E}+\int_0^1 D_{i, t}^{R E} d i-P_t^{R E} a\left(u_t^{R E}\right) K_{t-1}^\theta .
\end{aligned}
$$

The law of motion for capital is given by
$$
K_t^{R E}=(1-\delta) K_{t-1}^\theta+\left\{1-(\kappa / 2)\left(I_t^{R E} / I_{t-1}^\theta-\gamma\right)^2\right\} I_t^{R E},
$$
while the labour demand curve is simply $N_{h, t}^{R E}=N_t^{R E}\left(W_{h, t}^{R E} / W_t^{R E}\right)^{-\lambda_n /\left(\lambda_n-1\right)}$.


## Firm

The final output is produced by a perfectly competitive representative firm that combines a continuum of intermediate goods $Y_{i, t}^\theta$ using the technology:
$$
Y_t^\theta=\left[\int_0^1\left(Y_{i, t}^\theta\right)^{\frac{1}{\lambda_f}} d i\right]^{\lambda_f},
$$
where $\lambda_f$ controls the steady-state markup. Intermediate goods firms' production function is $Y_{i, t}^\theta=\left(u_{i, t}^\theta K_{i, t}^\theta\right)^\alpha\left(\gamma^t N_{i, t}^\theta\right)^{1-\alpha}$, where $K_{i, t}^\theta$ and $N_{i, t}^\theta$ are the capital and labour employed by firm $i$. From the cost minimization problem, the real marginal cost is given by
$$
M C_t^\theta=\frac{\left(R_t^{k, \theta}\right)^\alpha\left(W_t^\theta / P_t^\theta\right)^{1-\alpha}}{\alpha^\alpha(1-\alpha)^{1-\alpha} u_{i, t}^\theta\left(\gamma^t\right)^{1-\alpha}} .
$$

Intermediate firms face an adjustment cost à-la Rotemberg (1982) in changing their nominal price. Their problem is to choose $P_{i, t}^\theta$ to maximize
$$
\left(C_t^\theta-b C_{t-1}^\theta\right)^{-1} D_{i, t}^\theta / P_t^\theta+\beta \mathbb{E}_t^\theta \mathcal{V}_f\left(P_{i, t}^\theta\right)
$$

## Market clearing and monetary policy: 

The resource constraint is given by
$$
C_t^\theta+I_t^\theta+\left(\varphi_p / 2\right)\left(\Pi_t^\theta-\Pi\right)^2 Y_t^\theta+\left(\varphi_w / 2\right)\left(\Pi_{w, t}^\theta-\gamma \Pi\right)^2 W_t^\theta / P_t^\theta+a\left(u_t^\theta\right) K_{t-1}^\theta=Y_t^\theta,
$$
where $\Pi_{w, t}^\theta \equiv W_t^\theta / W_{t-1}^\theta$ is nominal wage inflation. The central bank follows a Taylor rule:
$$
R_t / R^\theta=\left(R_{t-1}^\theta / R\right)^{\rho_R}\left\{\left(\widetilde{\Pi}_t^\theta / \Pi\right)^{\phi_\pi}\left(Y_t^{G, \theta} /\left(\gamma Y_{t-1}^{G, \theta}\right)\right)^{\phi_Y}\right\}^{1-\rho_R} \exp \left(\varepsilon_t\right), \quad \varepsilon_t \sim N\left(0, \sigma_R^2\right),
$$
where $\tilde{\Pi}_t^\theta \equiv\left[\Pi_t^\theta \Pi_{t-1}^\theta \Pi_{t-2}^\theta \Pi_{t-3}^\theta\right]^{1 / 4}$ is the annual inflation, $\varepsilon_t$ is the iid monetary policy shock, and gross domestic product (GDP) is defined as $Y_t^{G, \theta} \equiv C_t^\theta+I_t^\theta$. We provide the equilibrium conditions in Supplementary Material, Appendix C.

In [ ]:
# Fixed parameters
gamma = 1.004                # quarterly deterministic growth rate
alpha = 0.3                  # Capital share
beta = 0.99                  # Discount factor
delta = 0.025                # Depreciation
lam_f = 1.1                  # Steady-state markup
pi_st = 4                    # Steady-state inflation
rho_A = 0.95                 # TFP Ar(1) persistence
rho_d = 0.95                 # Intertemporal pref Ar(1) persistence
rho_psi = 0.95               # Labor supply Ar(1) persistence
rho_zeta = 0.95              # Marginal efficiency of investment Ar(1) persistence
rho_g = 0.95                 # Government spending Ar(1) persistence
g_st = 0                     # Steady-state government spending to output ratio
J = 32                       # J
nu = 0                       # Partial indexation parameter (nu=0: no indexation to past inflation, nu=1: full indexation to past inflation)
rho_lam_f = 0.95             # Markup shock Ar(1) persistence
rho_x = 0.95                 # Risk premium shock Ar(1) persistence
lam_n = 1.1                # Steady-state wage markup
nu_w = 0                     # Partial wage indexation parameter (nu_w=0: no indexation to past wage inflation, nu_w=1: full indexation to past wage inflation)
dummyw = 1                   # Dummy parameter
bigstd_A = 0                 # Big standard deviation for A

fixedparam = [gamma, alpha, beta, delta, lam_f, pi_st, rho_A, rho_d, rho_psi, rho_zeta, rho_g, g_st, J, nu, rho_lam_f, rho_x, lam_n, nu_w, dummyw, bigstd_A]

'''
# Estimated parameters
param = np.zeros(13)
eta = param[0]               # Inverse Frisch elasticity
kappa = param[1]             # Investment adjustment cost
varphi = param[2]            # Price adjustment cost
rho_nr = param[3]            # Taylor rule smoothing
phi_pi = param[4]            # Taylor rule inflation
phi_dy = param[5]            # Taylor rule output
theta = param[6]             # Diagnostic theta weight
mean_Beta = param[7]         # Mean on Beta density
std_Beta = param[8]          # Std on Beta density
b = param[9]                 # External habit parameter
varphi_w = param[10]         # Wage adjustment cost
tau = param[11]              # Convexity of capital utilization cost (low tau: more elastic utilization)
bigstd_nr = param[12]        # Big standard deviation for non-recursive parameter
'''

#Estimated parameters

eta = 2.1065    # Inverse Frisch elasticity
kappa = 2.9696  # Investment adjustment cost
varphi = 195.3871   # Price adjustment cost
rho_nr = 0.0087 # Taylor rule smoothing
phi_pi = 1.0000 # Taylor rule inflation
phi_dy = 0.6696 # Taylor rule output
theta = 1.9655  # Diagnostic theta weight
mean_Beta = 0.1735  # Mean on Beta density
std_Beta = 0.0337   # Std on Beta density
b = 0.8005  # External habit parameter
varphi_w = 88.6210  # Wage adjustment cost
tau = 0.2229    # Convexity of capital utilization cost (low tau: more elastic utilization)
bigstd_nr = 0.1544  # Big standard deviation for non-recursive parameter

n_ss = 40                    # Number of steady states

# Create a list of parameters
p_st = [
    gamma, eta, alpha, beta, delta, 
    kappa, lam_f, varphi, rho_nr, phi_pi,
    phi_dy, pi_st, rho_A, rho_d, rho_psi, 
    rho_zeta, rho_g, g_st, theta, J, 
    mean_Beta, std_Beta, b, nu, rho_lam_f, 
    rho_x, lam_n, varphi_w, nu_w, tau
]



In [ ]:
# Initialize list to store variable names
snames = [None] * 200
ii = 0

# Variables without expectation
vc = ii
snames[ii] = 'C_t'  # [1] Consumption
ii += 1

vk = ii
snames[ii] = 'K_t'  # [2] Capital
ii += 1

vn = ii
snames[ii] = 'N_t'  # [3] Hours
ii += 1

vy = ii
snames[ii] = 'Y_t'  # [4] Output
ii += 1

vi = ii
snames[ii] = 'I_t'  # [5] Investment
ii += 1

vw = ii
snames[ii] = 'w_t'  # [6] Real wage
ii += 1

vrentk = ii
snames[ii] = 'R_t^k'  # [7] Rental rate of capital
ii += 1

vq = ii
snames[ii] = 'Q_t'  # [8] Marginal utility of consumption divided by the price level
ii += 1

vpi = ii
snames[ii] = '\pi_t'  # [9] Inflation
ii += 1

vpiw = ii
snames[ii] = '\pi_{w,t}'  # [10] Nominal wage inflation
ii += 1

vnr = ii
snames[ii] = 'R_t'  # [11] Nominal rate
ii += 1

vmc = ii
snames[ii] = 'MC_t'  # [12] Real marginal cost
ii += 1

vdeltai = ii
snames[ii] = '\Delta I_t'  # [13] Investment growth
ii += 1

vmu = ii
snames[ii] = '\mu_t'  # [14] Lagrangian multiplier on capital accumulation equation
ii += 1

vrrate = ii
snames[ii] = 'R_t-E_t^{\theta}\pi_{t+1}^{RE}'  # [15] Real rate
ii += 1

vlambda = ii
snames[ii] = '\lambda_t'  # [16] Marginal utility of consumption
ii += 1

vdeltapi = ii
snames[ii] = '\pi_t - \nu \pi_{t-1}'  # [17] Inflation minus lagged inflation
ii += 1

vdeltapiw = ii
snames[ii] = '\pi_{w,t} - \nu_w \pi_{w,t-1}'  # [18] Nominal wage inflation minus lagged nominal inflation
ii += 1

vu = ii
snames[ii] = 'u_t'  # [19] Capital utilziation rate
ii += 1

vgdp = ii
snames[ii] = 'GDP_t'  # [20] GDP
ii += 1

vpicuml = ii
snames[ii] = '\pi_{t,t-3}'  # [21] Cumulative inflation
ii += 1

vpilag = ii
snames[ii] = '\pi_{t-1}'  # [22] Lagged inflation
ii += 1

vpilag2 = ii
snames[ii] = '\pi_{t-2}'  # [23] Lagged inflation 2
ii += 1

vklag = ii
snames[ii] = 'K_{t-1}'  # [24] Lagged capital
ii += 1

vklag2 = ii
snames[ii] = 'K_{t-2}'  # [25] Lagged capital 2
ii += 1

vgdpgrowth = ii
snames[ii] = '\Delta GDP_t'  # [26] GDP growth
ii += 1

vA = ii
snames[ii] = 'A_t'  # [27] TFP shock
ii += 1

vd = ii
snames[ii] = 'd_t'  # [28] Intertemporal preference shock
ii += 1

vpsi = ii
snames[ii] = '\psi_t'  # [29] Labor supply shock
ii += 1

vzeta = ii
snames[ii] = '\zeta_t'  # [30] Marginal efficiency of investment shock
ii += 1

vg = ii
snames[ii] = 'g_t'  # [31] Government spending shock
ii += 1

vlam_f = ii
snames[ii] = '\lambda_{f,t}'  # [32] Markup shock
ii += 1

vx = ii
snames[ii] = 'x_t'  # [33] Risk premium shock
ii += 1

# Expectation variables
jj = 0

vElambda = ii
snames[ii] = 'E_t^{\theta}(\lambda_{t+1}^{RE})'  # [34] Expected marginal utility of consumption (RE or DE depending on position)
ii += 1
jj += 1

vErentk = ii
snames[ii] = 'E_t^{\theta}(R^k_{t+1}^{RE})'  # [35] Expected rental rate (RE or DE depending on position)
ii += 1
jj += 1

vEq = ii
snames[ii] = 'E_t^{\theta}(Q_{t+1}^{RE})'  # [36] Expected q (RE or DE depending on position)
ii += 1
jj += 1

vEdeltapi = ii
snames[ii] = 'E_t^{\theta}(\pi_{t+1}^{RE}-\nu\pi_{t}^{\theta})'  # [37] Expected inflation (RE or DE depending on position)
ii += 1
jj += 1

vEdeltapiw = ii
snames[ii] = 'E_t^{\theta}(\pi_{w,t+1}^{RE}-\nu_w \pi_{w,t}^{\theta})'  # [38] Expected wage inflation (RE or DE depending on position)
ii += 1
jj += 1

vEnr = ii
snames[ii] = 'E_t^{\theta}(R_{t+1}^{RE})'  # [39] Expected nominal interest rate (RE or DE depending on position)
ii += 1
jj += 1

vEdeltai = ii
snames[ii] = 'E_t^{\theta}(\Delta I_{t+1}^{RE})'  # [40] Expected investment growth (RE or DE depending on position)
ii += 1
jj += 1

vEmu = ii
snames[ii] = 'E_t^{\theta}(\mu_{t+1}^{RE})'  # [41] Expected lagrange multiplier on capital accumulation eq (RE or DE depending on position)
ii += 1
jj += 1

vEc = ii
snames[ii] = 'E_t^{\theta}(C_{t+1}^{RE})'  # [42] Expected consumotion (RE or DE depending on position)
ii += 1
jj += 1

vEpi = ii
snames[ii] = 'E_t^{\theta}(\pi_{t+1}^{RE})'  # [43] Expected inflation (RE or DE depending on position)
ii += 1
jj += 1

vEpicuml = ii
snames[ii] = 'E_t^{\theta}(\pi_{t+1,t-2}^{RE})'  # [44] Expected cumulative inflation (RE or DE depending on position)
ii += 1
jj += 1

vEgdpgrowth = ii
snames[ii] = 'E_t^{\theta} \Delta GDP_{t+1}'  # [45] Expected GDP growth (RE or DE depending on position)
ii += 1
jj += 1

vEklag2 = ii
snames[ii] = 'E_t^{\theta} K_{t-1}'  # [46] Expected lagged capital 2 (RE or DE depending on position)
ii += 1
jj += 1

In [ ]:
nst=1

# MS shock
czetaZ = p_st[20]  # Discrete shock

if nst > 1:
    if np.sum(p_st[40:50] != 0) != np.sum(p_st[30:40] != 0):
        raise ValueError("Parameters Error")
    # Expected intercepts for the shocks
    EczetaD = p_st[44, 0]

# Other parameters

# compute SS
rentk_st = gamma / beta + delta - 1
mc_st = 1 / lam_f
myOmega = (rentk_st / (mc_st * alpha)) ** (1 / (alpha - 1))
n_st = ((1 - b * gamma ** (-1)) ** (-1) * ((1 - g_st) * myOmega ** alpha - (gamma + delta - 1) * myOmega) ** (-1) * mc_st * (1 - alpha) * myOmega ** alpha * lam_n ** (-1)) ** (1 / (1 + eta))
c_st = ((1 - g_st) * myOmega ** alpha - (gamma + delta - 1) * myOmega) * n_st
k_st = myOmega * n_st
i_st = (gamma + delta - 1) * k_st
y_st = k_st ** alpha * n_st ** (1 - alpha)
w_st = mc_st * (1 - alpha) * y_st / n_st


# Dummy Variable
if nst > 1:
    ves1 = ii
    snames[ii] = 'ves1'
    ii += 1

# --------------------------------
nend = jj
neq = ii

snames = snames[0:neq+1]

# --------------------------------

# Exog shocks
ii = 0
eA = ii; ii += 1  # [1] TFP shock
enr = ii; ii += 1  # [2] monetary policy shock
ed = ii; ii += 1  # [3] intertemporal preference shock
epsi = ii; ii += 1  # [4] labor supply shock
ezeta = ii; ii += 1  # [5] marginal efficiency of investment shock
eg = ii; ii += 1  # [6] government spending shock
elam_f = ii; ii += 1  # [7] markup shock
ex = ii; ii += 1  # [8] risk premium shock

if nst > 1:
    sh_es1 = ii; ii += 1

nex = ii



In [ ]:
# Initialization of matrices
GAM0 = np.zeros((neq, neq))
GAM1 = np.zeros((neq, neq))
PSI = np.zeros((neq, nex))
PPI = np.zeros((neq, nend))

# Equation setup

# -----------------------------
eq = 0

# Loglinearized capital Euler Equation
GAM0[eq, vmu] = gamma
GAM0[eq, vElambda] = -beta * rentk_st
GAM0[eq, vErentk] = -beta * rentk_st
GAM0[eq, vEmu] = -beta * (1 - delta)

# Capital utilization choice
eq += 1
GAM0[eq, vrentk] = 1
GAM0[eq, vu] = -tau

# Loglinearized investment FONC
eq += 1
GAM0[eq, vlambda] = 1
GAM0[eq, vmu] = -1
GAM0[eq, vzeta] = -1
GAM0[eq, vdeltai] = kappa * gamma**2
GAM0[eq, vEdeltai] = -beta * kappa * gamma**2

# Investment growth
eq += 1
GAM0[eq, vdeltai] = 1
GAM0[eq, vi] = -1
GAM1[eq, vi] = -1

# Loglinearized bond Euler Equation
eq += 1

# specification 2
GAM0[eq, vq] = 1
GAM0[eq, vx] = -1
GAM0[eq, vEq] = -1
GAM0[eq, vnr] = -1

# Law of motion of Q
eq += 1
GAM0[eq, vq] = 1
GAM1[eq, vq] = 1
GAM0[eq, vpi] = 1
GAM0[eq, vlambda] = -1
GAM1[eq, vlambda] = -1

# Law of motion for capital
eq += 1
GAM0[eq, vk] = gamma
GAM0[eq, vi] = -(gamma + delta - 1)
GAM0[eq, vzeta] = -(gamma + delta - 1)
GAM1[eq, vk] = (1 - delta)

# Real wage
eq += 1
GAM0[eq, vw] = 1
GAM0[eq, vmc] = -1
GAM0[eq, vy] = -1
GAM0[eq, vn] = 1

# Rental rate of capital
eq += 1
GAM0[eq, vrentk] = 1
GAM0[eq, vmc] = -1
GAM0[eq, vy] = -1
GAM1[eq, vk] = -1

# Production function
eq += 1
GAM0[eq, vy] = 1
GAM0[eq, vA] = -1
GAM0[eq, vn] = -(1 - alpha)
GAM1[eq, vk] = alpha
GAM0[eq, vu] = -alpha

# Optimal price
eq += 1
GAM0[eq, vdeltapi] = 1
GAM0[eq, vEdeltapi] = -beta
GAM0[eq, vmc] = -1 / ((lam_f - 1) * varphi * pi_st**2)
GAM0[eq, vlam_f] = -1 / ((lam_f - 1) * varphi * pi_st**2)

# Optimal wage
eq += 1
GAM0[eq, vdeltapiw] = 1
GAM0[eq, vEdeltapiw] = -beta
GAM0[eq, vc] = -n_st / (varphi_w * gamma**2 * pi_st**2 * (lam_n - 1) * (1 - b * gamma**(-1)))
GAM1[eq, vc] = -b * gamma**(-1) * n_st / (varphi_w * gamma**2 * pi_st**2 * (lam_n - 1) * (1 - b * gamma**(-1)))
GAM0[eq, vpsi] = -n_st / (varphi_w * gamma**2 * pi_st**2 * (lam_n - 1))
GAM0[eq, vn] = -eta * n_st / (varphi_w * gamma**2 * pi_st**2 * (lam_n - 1))
GAM0[eq, vw] = n_st / (varphi_w * gamma**2 * pi_st**2 * (lam_n - 1))

# deltapi
eq += 1
GAM0[eq, vdeltapi] = 1
GAM0[eq, vpi] = -1
GAM1[eq, vpi] = -nu

# deltapiw
eq += 1
GAM0[eq, vdeltapiw] = 1
GAM0[eq, vpiw] = -1
GAM1[eq, vpiw] = -nu_w

# nominal wage inflation
eq += 1
GAM0[eq, vpiw] = 1
GAM0[eq, vpi] = -1
GAM0[eq, vw] = -1
GAM1[eq, vw] = -1

# Resource constraint
eq += 1
GAM0[eq, vc] = c_st / y_st
GAM0[eq, vi] = i_st / y_st
GAM0[eq, vg] = g_st
GAM0[eq, vy] = -(1 - g_st)
GAM0[eq, vu] = rentk_st * k_st / y_st

# GDP
eq += 1
GAM0[eq, vgdp] = 1
GAM0[eq, vy] = -1
GAM0[eq, vu] = rentk_st * k_st / y_st

# Taylor rule
eq += 1
GAM0[eq, vnr] = 1
GAM1[eq, vnr] = rho_nr
GAM0[eq, vpicuml] = -(1 - rho_nr) * phi_pi
GAM0[eq, vgdpgrowth] = -(1 - rho_nr) * phi_dy
PSI[eq, enr] = 1

# Cumulative inflation
eq += 1
GAM0[eq, vpicuml] = 1
GAM0[eq, vpi] = -0.25
GAM0[eq, vpilag] = -0.25
GAM0[eq, vpilag2] = -0.25
GAM1[eq, vpilag2] = 0.25

# Lagged inflation
eq += 1
GAM0[eq, vpilag] = 1
GAM1[eq, vpi] = 1

# Lagged inflation 2
eq += 1
GAM0[eq, vpilag2] = 1
GAM1[eq, vpilag] = 1

# Lagged capital
eq += 1
GAM0[eq, vklag] = 1
GAM1[eq, vk] = 1

# Lagged capital 2
eq += 1
GAM0[eq, vklag2] = 1
GAM1[eq, vklag] = 1

# Marginal utility of consumption
eq += 1
GAM0[eq, vlambda] = 1
GAM0[eq, vd] = -1
GAM0[eq, vc] = 1 / (1 - b * gamma**(-1))
GAM1[eq, vc] = 1 / (1 - b * gamma**(-1)) * b * gamma**(-1)

# Real rate
eq += 1
GAM0[eq, vrrate] = 1
GAM0[eq, vnr] = -1
GAM0[eq, vEpi] = 1

# GDP growth
eq += 1
GAM0[eq, vgdpgrowth] = 1
GAM0[eq, vgdp] = -1
GAM1[eq, vgdp] = -1

# TFP shock
eq += 1
GAM0[eq, vA] = 1
GAM1[eq, vA] = rho_A
PSI[eq, eA] = 1

# Intertemporal preference shock
eq += 1
GAM0[eq, vd] = 1
GAM1[eq, vd] = rho_d
PSI[eq, ed] = 1

# Labor supply shock
eq += 1
GAM0[eq, vpsi] = 1
GAM1[eq, vpsi] = rho_psi
PSI[eq, epsi] = 1

# Marginal efficiency of investment shock
eq += 1
GAM0[eq, vzeta] = 1
GAM1[eq, vzeta] = rho_zeta
PSI[eq, ezeta] = 1

# Government spending shock
eq += 1
GAM0[eq, vg] = 1
GAM1[eq, vg] = rho_g
PSI[eq, eg] = 1

# Markup shock
eq += 1
GAM0[eq, vlam_f] = 1
GAM1[eq, vlam_f] = rho_lam_f
PSI[eq, elam_f] = 1

# Risk premium shock
eq += 1
GAM0[eq, vx] = 1
GAM1[eq, vx] = rho_x
PSI[eq, ex] = 1

# Expectational errors
eer = 0

eq += 1
GAM0[eq, vlambda] = 1
GAM1[eq, vElambda] = 1
PPI[eq, eer] = 1

eer += 1
eq += 1
GAM0[eq, vrentk] = 1
GAM1[eq, vErentk] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vq] = 1
GAM1[eq, vEq] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vdeltapi] = 1
GAM1[eq, vEdeltapi] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vdeltapiw] = 1
GAM1[eq, vEdeltapiw] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vnr] = 1
GAM1[eq, vEnr] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vdeltai] = 1
GAM1[eq, vEdeltai] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vmu] = 1
GAM1[eq, vEmu] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vc] = 1
GAM1[eq, vEc] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vpi] = 1
GAM1[eq, vEpi] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vpicuml] = 1
GAM1[eq, vEpicuml] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vgdpgrowth] = 1
GAM1[eq, vEgdpgrowth] = 1
PPI[eq, eer] = 1

eq += 1
eer += 1
GAM0[eq, vklag2] = 1
GAM1[eq, vEklag2] = 1
PPI[eq, eer] = 1



In [ ]:
TT, CC, RR, _, _, _, _, RC = gensysct(GAM0,GAM1,0,PSI,PPI)

## Solve DE transition rule

In [ ]:
nst = 1

p_ss = []
p_er = 1
o_er = 0
HB = 1
MAT = []

msel = np.zeros(40)
msel[14] = 1
msel[12] = 33
MAT.append([])

msel[9] = 1

nS = TT.shape[0]
DD = np.zeros((nS, 1))
ZZ = np.eye(nS)
CCtheta = np.zeros((nS, nst))
nsh = RR.shape[1]
QQ = p_er ** 2 * np.eye(nsh)
Jalt = 2
dummywalt = 0

In [ ]:
nE = 13  # number of expectational variables


# define selection matrix M
M = np.zeros((nE, nS))
M[0, vlambda] = 1  # lambda
M[1, vrentk] = 1  # rentk
M[2, vq] = 1  # q
M[3, vdeltapi] = 1  # deltapi
M[4, vdeltapiw] = 1  # deltapiw
M[5, vnr] = 1  # nr
M[6, vdeltai] = 1  # deltai
M[7, vmu] = 1  # mu
M[8, vc] = 1  # c
M[9, vpi] = 1  # pi
M[10, vpicuml] = 1  # picuml
M[11, vgdpgrowth] = 1  # gdpgrowth
M[12, vklag2] = 1  # lagges capital 2

In [ ]:
def myBetapdf(x, mu, sig):
    # 计算参数 alpha 和 beta，用于参数化 Beta 分布
    alpha_val = mu * (mu * (1 - mu) / sig ** 2 - 1)
    beta_val = (1 - mu) * (mu * (1 - mu) / sig ** 2 - 1)
    
    # 评估概率密度函数
    myBetapdf = scipy.stats.beta.pdf(x, alpha_val, beta_val)
    
    return myBetapdf

In [ ]:
def reduce_dim_2(TTi, RRi, ZZi, CCi, snames=None):
    # This function reduces the size of the matrices characterizing the solution of a DSGE or MS-DSGE model.
    # The solution is written as 
    # yt = ZZ * St                                  Observation equation
    # St = CC(ir) + TT(ir)*St-1 + RR * QQ * epst    Transition equation
    # SS is a selection matrix relates original state vector St with reduced dimension state vector Stilt
    # Stilt = SS*St

    # Get the number of variables
    N = TTi.shape[1]

    # Calculate the row sums of absolute values of TT and ZZ matrices
    Tx = np.sum(np.abs(TTi), axis=1)
    Zx = np.sum(np.abs(ZZi), axis=1)

    # Find non-zero indices for TT and ZZ
    indz = np.where(np.sum(np.abs(Zx), axis=0) > 1e-9)[0]
    indt = np.where(np.sum(np.abs(Tx), axis=0) > 1e-9)[0]

    # Take union of indz and indt
    uni_z_t = np.union1d(indz, indt)

    # Reduce the matrices based on non-zero indices
    ZZ = ZZi
    RR = RRi
    TT = TTi
    CC = CCi

    Ntil = len(uni_z_t)
    SS = np.zeros((Ntil, N))

    for i in range(Ntil):
        SS[i, uni_z_t[i]] = 1

    # Adjust snames if it's provided
    if snames is not None:
        if snames.shape[1] == 2:
            snames[0] = snames[0][:, uni_z_t]
        else:
            snames = snames[:, uni_z_t]
    else:
        snames = []

    return TT, RR, ZZ, CC, SS, snames


In [ ]:
# weights on lagged expectations
myweight = np.zeros((J, 1))
for kk in range(1, J + 1):
    myweight[kk - 1, 0] = myBetapdf(kk / (J + 1), mean_Beta, std_Beta)

myweight = myweight / np.sum(myweight)

'''
if len(args) > 13:
    if dummyw == 0:
        myweight = np.zeros((J, 1))
        myweight[J - 1, 0] = 1
'''
GAM0theta = np.zeros((nS, nS))
GAM1theta = np.zeros((nS, nS))
GAM2theta = np.zeros((nS, nE))
PSItheta = np.zeros((nS, nsh))

GAM011 = GAM0[:nS - nE, :nS - nE]
GAM012 = GAM0[:nS - nE, nS - nE:]

GAM111 = GAM1[:nS - nE, :nS - nE]
GAM112 = GAM1[:nS - nE, nS - nE:]

TT11 = TT[:nS - nE, :nS - nE]
TT12 = TT[:nS - nE, nS - nE:]
TT21 = TT[nS - nE:, :nS - nE]
TT22 = TT[nS - nE:, nS - nE:]

PSI1 = PSI[:nS - nE, :]

GAM0theta[:nS - nE, :nS - nE] = GAM011

M1 = M[:, :nS - nE]
GAM0theta[nS - nE:, :nS - nE] = -M1 @ TT11
GAM0theta[nS - nE:, nS - nE:] = np.eye(nE) - M1 @ TT12

GAM2theta[:nS - nE, :] = -GAM012

GAM1theta[:nS - nE, :nS - nE] = GAM111
GAM1theta[:nS - nE, nS - nE:] = GAM112

PSItheta[:nS - nE, :] = PSI1

A0theta = GAM0theta - (1 + theta)* GAM2theta @ M @ TT

TTtil, RRtil, Mtil, CCtheta, SS, _ = reduce_dim_2(TT, RR, M, np.zeros((nS, 1)))

nStil = TTtil.shape[1]

TTtheta = np.zeros((nS + J * nStil, nS + J * nStil))
RRtheta = np.zeros((nS + J * nStil, nsh))

TTtheta[:nS, :nS] = np.linalg.inv(A0theta) @ (GAM1theta - theta * myweight[0] *  GAM2theta @ M @ TT ** 2)

if J > 1:
    for jj in range(2, J + 1):
        TTtheta[:nS, nS + (jj - 2) * nStil:nS + (jj - 1) * nStil] = -np.linalg.inv(
            A0theta) @ (  theta * myweight[jj - 1] * GAM2theta @ Mtil @ TTtil ** (jj + 1))

if J > 1:
    TTtheta[nS:nS + nStil, :nS] = SS
    if J > 2:
        for jj in range(3, J + 1):
            TTtheta[nS + (jj - 2) * nStil:nS + (jj - 1) * nStil,
            nS + (jj - 3) * nStil:nS + (jj - 2) * nStil] = np.eye(nStil)

RRtheta[:nS, :] = np.linalg.inv(A0theta) @ PSItheta

# let v_t be a vector of observables; v_t = x_t so that v_t = A z_t where
A = np.zeros((nS, nS + J * nStil))
A[:, :nS] = np.eye(nS)

# compute up to K-step-ahead DE forecasts
K = 4

B = np.zeros((nS + K * nE, nS + J * nStil))
for kk in range(1, K + 1):
    B[nS + (kk - 1) * nE:nS + kk * nE, :nS] = (1 + theta) * M @ TT ** kk
    for jj in range(1, J + 1):
        B[nS + (kk - 1) * nE:nS + kk * nE, nS + (jj - 1) * nStil:nS + jj * nStil] = -theta * myweight[jj - 1] * Mtil @ TTtil ** (kk + jj)

# also compute forecasts under equilibrium (DE) law of motion
P = np.zeros((nE, nS + J * nStil))
P[:, :nS] = M

C = np.zeros((nS + K * nE + K * nE, nS + J * nStil))
C[:nS + K * nE, :] = B

for kk in range(1, K + 1):
    C[nS + K * nE + (kk - 1) * nE:nS + K * nE + kk * nE, :] = P @ TTtheta ** kk
